# Experimenting with the dcrhino3 feature extraction

Trying to understand the functionality better and looking into how to add my own (bruno) feature extraction functions.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from matplotlib import pyplot as plt

os.chdir('/home/bruno/datacloud')

plt.style.use('seaborn-dark')

plt.rcParams['axes.grid'] = True
plt.rcParams['figure.figsize'] = (15, 5)

In [ ]:
from dcrhino3.feature_extraction.feature_extractor_j2 import FeatureExtractorJ2
from dcrhino3.helpers.general_helper_functions import dict_to_object

# Define Picks and Window Widths 

I copied this from a process flow

In [ ]:
transformed_args = {
        "additional_pick_based_amplitude_windows": {
          "axial": {
            "multiple_1": {
              "left_upper_bound_offset": -0.002,
              "left_lower_bound_offset": -0.006,
              "right_upper_bound_offset": 0.006,
              "right_lower_bound_offset": 0.002
            },
            "primary": {
              "left_upper_bound_offset": -0.002,
              "left_lower_bound_offset": -0.006,
              "right_upper_bound_offset": 0.006,
              "right_lower_bound_offset": 0.002
            }
          },
          "tangential": {
            "multiple_1": {
              "left_upper_bound_offset": -0.002,
              "left_lower_bound_offset": -0.006,
              "right_upper_bound_offset": 0.006,
              "right_lower_bound_offset": 0.002
            },
            "primary": {
              "left_upper_bound_offset": -0.002,
              "left_lower_bound_offset": -0.006,
              "right_upper_bound_offset": 0.006,
              "right_lower_bound_offset": 0.002
            }
          }
        },
        "ACOUSTIC_VELOCITY": 4755.,
        "amplitude_half_widths": {
          "axial": {
            "multiple_3": 0.00105,
            "multiple_2": 0.00105,
            "multiple_1": 0.00105,
            "primary": 0.00105
          },
          "tangential": {
            "multiple_3": 0.00105,
            "multiple_2": 0.00105,
            "multiple_1": 0.00105,
            "primary": 0.00105
          }
        },
        "sensor_saturation_g": 1000,
        "manual_time_windows": {
          "axial": {
            "multiple_3": [-0.004, 0.005],
            "multiple_2": [-0.004, 0.005],
            "multiple_1": [-0.004, 0.005],
            "primary": [-0.004,0.005]
          },
          "tangential": {
            "multiple_3": [-0.004, 0.005],
            "multiple_2": [-0.004, 0.005],
            "multiple_1": [-0.004, 0.005],
            "primary": [-0.004,0.005]
          }
        },
        "sensor_distance_to_shocksub": 1,
        "upsample_sampling_rate": 10000,
        "SHEAR_VELOCITY": 2654.,
        "time_picks": {
          "axial": {
            "multiple_3": "maximum",
            "multiple_2": "maximum",
            "multiple_1": "maximum",
            "primary": "maximum"
          },
          "tangential": {
            "multiple_3": "maximum",
            "multiple_2": "maximum",
            "multiple_1": "maximum",
            "primary": "maximum"
          }
        },
        "amplitude_picks": {
          "axial": {
            "multiple_3": "integrated_absolute_amplitude",
            "multiple_2": "integrated_absolute_amplitude",
            "multiple_1": "integrated_absolute_amplitude",
            "primary": "integrated_absolute_amplitude"
          },
          "tangential": {
            "multiple_3": "integrated_absolute_amplitude",
            "multiple_2": "integrated_absolute_amplitude",
            "multiple_1": "integrated_absolute_amplitude",
            "primary": "integrated_absolute_amplitude"
          }
        }
      }

Here we need to convert the dictionary to dictionary object. 

In [ ]:
def _dict_to_object(obj):
    for k, v in obj.items():
        if isinstance(v, dict):
            item = _dict_to_object(v)
            obj[k] = dict_to_object(v)
    return dict_to_object(obj)

In [ ]:
transformed_args = _dict_to_object(transformed_args)

# Define a wavelet 

In [ ]:
import numpy as np

In [ ]:
from theory.core import TheoreticalWavelet, Pipe, Rock

In [ ]:
pipe = Pipe()

rock = Rock(2000, 2000)

theoretical = TheoreticalWavelet(pipe, rock, )

In [ ]:
wavelet = theoretical.primary_in_time_domain(500, filtered=True)
timestamp = theoretical.get_time_range_for_window(500) / 1000

In [ ]:
theoretical.sampling_rate

In [ ]:
plt.plot(timestamp, wavelet)

# Call feature extractor 

In [ ]:
fej2 = FeatureExtractorJ2('axial', trimmed_trace=wavelet, timestamp=timestamp/1000, transformed_args=transformed_args, sampling_rate=10000)

In [ ]:
ef = fej2.extract_features()

Seems to work, but I think the features are not correct. Ignore everything below, lets jam tmr.

In [ ]:
plt.plot(timestamp, wavelet)
plt.vlines(-1.90, -30000, 50000)
plt.vlines(2.95, -30000, 50000)
plt.vlines(4.70, -30000, 50000, colors='r')
plt.xlim(-10, 10)

In [ ]:
np.trapz(wavelet[(timestamp>=-1.9) & (timestamp<=2.95)])

In [ ]:
ef